# Structural analysis (interaction frequency) with MDAnalysis

In [2]:
import matplotlib.pyplot as plt
import re
import glob
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis import distances

/opt/homebrew/anaconda3/envs/IPL-workshop/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### First convert models into PDB

In [3]:
# AF3 models are stored in AF3_models directory as cif files
AF3_models=glob.glob("AF3_antibody/*.cif")
# convert to PDB
for f in AF3_models:
    # get PDB file name
    fpdb=re.sub(".cif",".pdb",f)
    # convert to PDB
    parser = PDB.MMCIFParser()
    structure = parser.get_structure(structure_id="PDB", filename=f)
    io=PDBIO()
    io.set_structure(structure)
    io.save(fpdb)

NameError: name 'PDB' is not defined

### List of PDBs to analyze

In [ ]:
# AF3 models are stored in AF3_models directory as pdb files
AF3_models=glob.glob("AF3_antibody/*.pdb")

### Set a reasonable distance cutoff for contact:

In [ ]:
distance_cutoff = 8.0

### For each model, identify interfacial residues

In [ ]:
# prepare dictionary
contacts={}
# loop over models
for PDB_ in AF3_models:
    # create universe
    u = mda.Universe(PDB_)
    # get all CA atoms
    CA_atoms = u.select_atoms('name CA')
    # get the CA atoms of spike and from the antibody Fab domain. You must edit the chains that must be selected if necessary
    spike_CA = CA_atoms.select_atoms('chainID A')
    # get the CA atoms of the antibody Fab domain. This all the other chains wrt to the spike chain
    antibody_CA = CA_atoms.select_atoms('not chainID A')
    # calculate distance matrix
    distance_matrix = distances.distance_array(spike_CA, antibody_CA)
    # identify contacts
    ids = np.where(distance_matrix < distance_cutoff)
    # contact residues
    contacts[PDB_] = sorted(list(set(ids[0])))

### Create a PDB file with only spike and interaction information in Bfactor column

In [ ]:
# use last universe: set all the bfactors to zero
atoms = u.select_atoms('chainID A')
for at in atoms:
    at.tempfactor = 0.0
# set bfactor equal to residue contact probability
for key in contacts:
    for id in contacts[key]:
        r = spike_CA[id].residue
        for at in r.atoms:
            at.tempfactor += 1.0
# normalize
for at in atoms:
    at.tempfactor /= float(len(AF3_models))
# save PDB file
atoms.write("spike_contacts.pdb")